In [59]:
import torch
import torch.nn as nn
import numpy as np
from sklearn.preprocessing import LabelEncoder
import torch.nn.functional as F

In [12]:
rna_sequnce = ['ACT','GTU' , 'AUN']

In [85]:
def encode_seq(kmer_token):

    # A 1 0 0 0
    # C 0 1 0 0
    # T/U 0 0 0 1
    # G 0 0 1 0
    # N 0 0 0 0

    encoding_dict = {
        'A': [1, 0, 0, 0],
        'C': [0, 1, 0, 0],
        'G': [0, 0, 1, 0],
        'T': [0, 0, 0, 1],
        'U': [0, 0, 0, 1],
        'N': [0, 0, 0, 0],
    }

    encoded_sequence = []
    for  base in kmer_token:
        encoded_sequence.append(encoding_dict[base])
    return np.array(encoded_sequence).flatten()

x = []
for seq in rna_sequnce:
    x.append(encode_seq(seq))
x = torch.from_numpy( np.array(x))
x

tensor([[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
        [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1],
        [1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]], dtype=torch.int32)

In [86]:
labels = ['red','blue','green']
le = LabelEncoder()
y = le.fit_transform(labels)
y = torch.from_numpy(np.array(y))
y 

tensor([2, 0, 1])

# Single Directional LSTM

In [87]:
hidden_dim = 3
embedding_dim = 7
input_dim = 12

embedding = nn.Embedding(input_dim, embedding_dim, padding_idx=0)
lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, num_layers=1, batch_first=True )
fc = nn.Linear(hidden_dim, 3) 

In [88]:
embedding_out =  embedding(x)
print(embedding_out.shape)

torch.Size([3, 12, 7])


In [89]:
lstm_out , (h,c) = lstm(embedding_out)

h[-1] # In case of Bi Directional use both output

tensor([[ 0.1908, -0.2606, -0.0924],
        [ 0.1917, -0.2623, -0.0830],
        [ 0.2672, -0.2596, -0.2928]], grad_fn=<SelectBackward0>)

In [37]:
outputs = fc(h[-1])
outputs

tensor([[ 0.0084,  0.2081, -0.5230],
        [ 0.0107,  0.2096, -0.5206],
        [-0.0017,  0.2090, -0.5449]], grad_fn=<AddmmBackward0>)

In [38]:
_, predicted = torch.max(outputs,1)
predicted

tensor([1, 1, 1])

In [44]:
loss_function = nn.CrossEntropyLoss()
loss = loss_function(outputs, y)
loss.item()

1.1403356790542603

# Bi-Directional LSTM with Attention

In [101]:
hidden_dim = 3
embedding_dim = 7
input_dim = 12

b_embedding = nn.Embedding(input_dim, embedding_dim, padding_idx=0)
b_lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, num_layers=1, batch_first=True , bidirectional=True )
b_fc = nn.Linear(2*hidden_dim, 3) 

In [103]:
b_embedding_out =  embedding(x)
print(b_embedding_out.shape)

torch.Size([3, 12, 7])


In [104]:
b_lstm_out , (h,c) = lstm(embedding_out)

#       [[[ 0.1242,  0.0155,  0.1971],
#          [ 0.1305,  0.0160,  0.2074],
#          [ 0.1432, -0.0180, -0.1529]],

#         [[ 0.0721, -0.3585, -0.2495],
#          [-0.0041, -0.4517, -0.1265],
#          [ 0.0710, -0.3557, -0.2356]]]

b_h = torch.cat((h[-2, :, :], h[-1, :, :]), dim=1)
b_h

tensor([[-0.3274,  0.3477,  0.0392,  0.2940, -0.1465, -0.0589],
        [-0.2995,  0.3459,  0.0437,  0.1803, -0.1686,  0.0204],
        [-0.4515,  0.2758, -0.1362,  0.2935, -0.1449, -0.0585]],
       grad_fn=<CatBackward0>)

In [116]:
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_dim):
        super(BahdanauAttention, self).__init__()
        self.W = nn.Linear(hidden_dim, hidden_dim)
        self.v = nn.Linear(hidden_dim, 1)

    def forward(self, hidden, encoder_outputs):
        hidden = hidden.unsqueeze(1)
        hidden = hidden.expand(-1, encoder_outputs.size(1), -1)
        # Aboe two steps makes hidden dim similar to encoder_outputs
        
        alignment_scores = self.v(torch.tanh(self.W(hidden + encoder_outputs)))
        attention_weights = F.softmax(alignment_scores, dim=1)
        context_vector = torch.sum(attention_weights * encoder_outputs, dim=1)
        return context_vector, attention_weights

In [118]:
attention = BahdanauAttention(2*hidden_dim)
context_vector , weights = attention(b_h , b_lstm_out)
context_vector

tensor([[-0.3244,  0.2820, -0.0638,  0.2010, -0.1485,  0.0028],
        [-0.3227,  0.2817, -0.0600,  0.2047, -0.1535, -0.0042],
        [-0.3397,  0.2763, -0.0792,  0.1829, -0.1382,  0.0202]],
       grad_fn=<SumBackward1>)

In [78]:
outputs = fc(context_vector)
outputs

tensor([[ 0.3206, -0.0334,  0.1136],
        [ 0.3347, -0.0262,  0.1085],
        [ 0.3144, -0.0332,  0.1178]], grad_fn=<AddmmBackward0>)

In [79]:
_, predicted = torch.max(outputs,1)
predicted

tensor([0, 0, 0])

In [80]:
loss_function = nn.CrossEntropyLoss()
loss = loss_function(outputs, y)
loss.item()

1.106108546257019